In [4]:
import cv2
import os
import numpy as np

In [5]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [9]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

from tensorflow.keras.models import load_model
model = load_model('C:/Users/USER/Desktop/Assignment 2 file/Code/run/Model 1')

video_capture = cv2.VideoCapture(0)

def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (255, 0, 0), 3)
        crop_image = vid[y:y+h, x:x+w]
        resize_img = cv2.resize(crop_image, (227, 227))
        resize_img = cv2.cvtColor(resize_img, cv2.COLOR_BGR2RGB)
        predictions = model.predict(np.array([resize_img]), verbose=0)
        binary_prediction = (predictions > 0.5).astype(int)
        
        label = "Mask Worn" if binary_prediction == 0 else "No Mask"
        color = (0, 255, 0) if binary_prediction == 0 else (255, 0, 0)
        
        cv2.putText(vid, label, (x, y+h+25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2, cv2.LINE_AA)
    
    return faces

while True:
    result, video_frame = video_capture.read()
    if not result:
        break

    faces = detect_bounding_box(video_frame)

    cv2.imshow("Face Mask Detection", video_frame)

    key = cv2.waitKey(10)
    if key == 27:  # The Esc key
        break

video_capture.release()
cv2.destroyAllWindows()
